# Testing, debugging, profiling

Dans cette section, nous allons voir comment tester, débugger et profiler du code Python. 

Cette section est adapté des [SciPy lecture notes](http://www.scipy-lectures.org/) et d'un [cours](https://paris-swc.github.io/python-testing-debugging-profiling/) de Marcel Stimberg, lui-même adapté d'un cours de [Kathryn Huff](http://katyhuff.github.io/python-testing/). 

## 1. Testing

### 1.1 Assertions

Les assertions sont les types de tests les plus simples. Elles permettent de vérifier la cohérence interne d'un code.  
En Python, une assertion a le comportement suivant:

In [ ]:
assert True == False

In [ ]:
assert True == True

L'assertion arrête l'exécution du code si la comparaison est fausse et ne fait rien si la comparaison est juste.  
C'est par exemple un outil pratique pour vérifier que l'input d'une fonction est correcte: 

In [ ]:
def mean(num_list):
    assert len(num_list) != 0
    return sum(num_list) / len(num_list)

In [ ]:
mean([])

L'avantage des assertions est leur facilité d'utilisation et leur compacité: une ligne de code! Le désavantage des assertions est qu'elles arrêtent l'exécution du code et ne produisent pas de message informatif à l'utilisateur. 

### 1.2 Exceptions

Vous avez peut être déjà rencontré une exception Python pendant la journée de hier. Une exception est levée quand on rencontre une erreur. On peut aussi lever des exceptions dans notre code. C'est par exemple pratique pour vérifier si l'entrée d'une fonction est de type attendu, si ce n'est pas le cas une exception peut être levée avec un message informatif.  

Considérons par exemple la fonction ci-dessous qui "rescale" un tableau numpy sur un intervalle donné: 

In [ ]:
import numpy as np

def rescale(data, lower=0.0, upper=1.0):
    """
    (Linearly) rescale the data so that it fits into the given range.

    Parameters
    ----------
    data : ndarray
        The data to rescale.
    lower : number, optional
        The lower bound for the data. Defaults to 0.
    upper : number, optional
        The upper bound for the data. Defaults to 1.

    Returns
    -------
    rescaled : ndarray
        The data rescaled between ``lower`` and ``upper``.
    """
    data_min = np.min(data)
    data_max = np.max(data)
    normalized_data = (data - data_min) / (data_max - data_min)
    rescaled_data = lower + (upper - lower) * normalized_data
    return rescaled_data

Que se passe-t-il si les éléments du tableau d'entrée ont tous la même valeur?

In [ ]:
rescale(np.array([0, 0, 0]))

Pour éviter cela, nous allons lever l'exception "Value Error" si les éléments du tableau d'entrée ont tous la même valeur. 
`ValueError` est une des classes d'erreurs intégrée dans Python, qui permet de signaler un argument qui a le bon type mais une valeur inappropriée.  

In [ ]:
def rescale(data, lower=0.0, upper=1.0):
    data_min = np.min(data)
    data_max = np.max(data)
    if not data_max > data_min:
        raise ValueError('Cannot rescale data: all values are identical.')
    normalized_data = (data - data_min) / (data_max - data_min)
    rescaled_data = lower + (upper - lower) * normalized_data
    return rescaled_data

In [ ]:
rescale(np.array([0, 0, 0]))

Il existe un certain nombre d'exceptions prédéfinies dans Python ([voir la liste ici](https://docs.python.org/2/library/exceptions.html)). 
Vous pouvez aussi définir vos propres exceptions. Elles doivent dériver de la classe `Exception` de Python ([plus de détails ici](https://docs.python.org/2/tutorial/errors.html#tut-userexceptions)).

Au lieu d'arrêter l'exécution du code quand une erreur est recontrée, on peut "attrapper" l'exception afin de modifier le comportement du code:

In [ ]:
def rescale(data, lower=0.0, upper=1.0):
    try:
        data_min = np.min(data)
    except ValueError:
        raise ValueError('Could not calculate the minimum of the input data -- maybe it is empty?')
    data_max = np.max(data)
    if not data_max > data_min:
        raise ValueError('Cannot rescale data: all values are identical.')
    normalized_data = (data - data_min) / (data_max - data_min)
    rescaled_data = lower + (upper - lower) * normalized_data
    return rescaled_data

In [ ]:
rescale(np.array([]))

In [ ]:
def rescale(data, lower=0.0, upper=1.0):
    try:
        data_min = np.min(data)
    except ValueError:
        return np.array([])
    data_max = np.max(data)
    if not data_max > data_min:
        raise ValueError('Cannot rescale data: all values are identical.')
    normalized_data = (data - data_min) / (data_max - data_min)
    rescaled_data = lower + (upper - lower) * normalized_data
    return rescaled_data

In [ ]:
rescale(np.array([]))

On peut aussi gérer plusieurs types d'exception:

In [ ]:
def rescale(data, lower=0.0, upper=1.0):
    try:
        data_min = np.min(data)
    except ValueError:
        return np.array([])
    except TypeError:
        raise TypeError('Can only rescale numerical data')
    data_max = np.max(data)
    if not data_max > data_min:
        raise ValueError('Cannot rescale data: all values are identical.')
    normalized_data = (data - data_min) / (data_max - data_min)
    rescaled_data = lower + (upper - lower) * normalized_data
    return rescaled_data

In [ ]:
rescale(np.array(['chien', 'chat']))

Quel est l'avantage d'utiliser `try/except` par rapport à une vérification explicite? Comparez ces deux fonctions qui renvoient le minimum et le maximum comme une tuple:  

In [ ]:
def minmax1(data):
   try:
       data_min = np.min(data)
       data_max = np.max(data)
   except TypeError:
       raise TypeError('Need numerical data.')
   return (data_min, data_max)

def minmax2(data):
   if not isinstance(data, np.ndarray):
       raise TypeError('Need numerical data.')
   data_min = np.min(data)
   data_max = np.max(data)
   return (data_min, data_max)

In [ ]:
l = [1, 2, 3]
print(minmax1(l))
print(minmax2(l))

### 1.3 Tests unitaires

Ce que nous venons de voir nous protége contre certaines erreurs, mais on ne peut pas savoir si notre code marche avant de l'utiliser... Heureusement, on peut écrire des tests unitaires!  
En Python, les tests unitaires sont typiquement des fonctions de tests qui appellent les fonctions dans le code de base et font des assertions. Pour faire tourner ces tests, un "framework" de test est souvent requit, il permet de collecter tous les tests et de les éxécuter.  
Nous allons d'abord écrire des tests pour notre fonction `rescale` et les exécuter individuellement. Nous introduirons ensuite le "framework" de test [`pytest`](http://pytest.org/latest/). Python fournit un framework de test intégré [`unittest`](https://docs.python.org/3/library/unittest.html). Toutefois, le framework pytest est plus simple d'utilisation.



Un test comporte générallement trois parties: **"Etant donné, quand, alors"**. Etant donné certaines données, quand telle action est réalisée, alors on attend ce résultat.

Note: Numpy fournit quelques fonctions utiles pour effectuer des tests dans le package [numpy.testing.utils](http://docs.scipy.org/doc/numpy-1.10.1/reference/routines.testing.html), comme `assert_allclose`.

In [ ]:
def rescale(data, lower=0.0, upper=1.0):
    try:
        data_min = np.min(data)
    except ValueError:
        return np.array([])
    except TypeError:
        raise TypeError('Can only rescale numerical data')
    data_max = np.max(data)
    if not data_max > data_min:
        raise ValueError('Cannot rescale data: all values are identical.')
    normalized_data = (data - data_min) / (data_max - data_min)
    rescaled_data = lower + 2 * (upper - lower) * normalized_data
    return rescaled_data

In [ ]:
from numpy.testing.utils import assert_allclose

def test_rescale():
    data = np.array([1.0, 2.0, 3.0])
    rescaled = rescale(data)
    expected = np.array([0.0, 0.5, 1.0])
    assert_allclose(rescaled, expected)

In [ ]:
test_rescale()

#### Tests unitaires avec Pytest

Pytest permet l'utilisation de la fonction standard `assert` et de ses dérivées ([plus de détails ici]()). 

Nous allons écrire une suite de tests pour la fonction `preprocess.py` et les exécuter grâce à `pytest`. Voici la fonction `preprocess.py`

In [1]:
!cat exo_testing/preprocess.py

cat: exo_testing/preprocess.py: No such file or directory


Nous allons écrire des tests pour cette fonction dans le fichier `test_whiten.py` (qui utilisent l'assertion `assert_allclose`):

In [ ]:
%%writefile exo_testing/test_whiten.py
from numpy.testing.utils import assert_allclose, assert_equal
import numpy

from preprocess import whiten

def test_1d():
    test_data = numpy.array([1, 3, 5, 7])
    whitened = whiten(test_data)
    assert_allclose(whitened.mean(), 0)
    assert_allclose(whitened.std(), 1)
    assert_allclose(whitened*test_data.std() + test_data.mean(),
                    test_data)
def test_2d():
    test_data = numpy.array([[1, 3, 5, 7],
                             [2, 3, 4, 1]])
    whitened = whiten(test_data)
    assert_allclose(whitened.mean(), 0)
    assert_allclose(whitened.std(), 1)
    assert_allclose(whitened*test_data.std() + test_data.mean(),
                    test_data)

On peut maintenant exécuter `pytest` (commande `py.test`) dans le dossier où se trouvent le fichier de test.

Le package pytest collecte les tests (fichiers `test_*`) dans le dossier, les éxécute et fournit un rapport de test.
Pour avoir plus/moins de détails sur les tests, on peut utiliser l'option `-v`/`-q` (pour `verbose`/`quiet`).

Nos test ne passent pas, car ils sont en fait trop strict! Le rapport nous informe que:
```
E       AssertionError: 
E       Not equal to tolerance rtol=1e-07, atol=0
```
La tolérance absolue est trop stricte.. On peut ajouter `atol=1e-15` à notre assertion pour résoudre ce problème.

In [ ]:
%%writefile exo_testing/test_whiten.py
from numpy.testing.utils import assert_allclose, assert_equal
import numpy

from preprocess import whiten

def test_1d():
    test_data = numpy.array([1, 3, 5, 7])
    whitened = whiten(test_data)
    assert_allclose(whitened.mean(), 0, atol=1e-15)
    assert_allclose(whitened.std(), 1, atol=1e-15)
    assert_allclose(whitened*test_data.std() + test_data.mean(),
                    test_data)
def test_2d():
    test_data = numpy.array([[1, 3, 5, 7],
                             [2, 3, 4, 1]])
    whitened = whiten(test_data)
    assert_allclose(whitened.mean(), 0, atol=1e-15)
    assert_allclose(whitened.std(), 1, atol=1e-15)
    assert_allclose(whitened*test_data.std() + test_data.mean(),
                    test_data)

## 2. Debugging

### 2.1 Post-mortem debugging

Python fournit un debugger intégré. Quand une exception est levée, son contexte est enregistré et peut être examiné. Ce type de debugging est dit "post-mortem", car on utilise le debugger après le crash du programme, par opposition à exécuter le programme sous le contrôle du debugger.

Voici du code avec des erreurs:

In [ ]:
import numpy as np
def find_first(data, element):
    """
    Return the index of the first appearance of ``element`` in
    ``data`` (or -1 if ``data`` does not contain ``element``).
    """
    counter = 0
    while counter <= len(data):
        if data[counter] == element:
            return counter
        counter += 1
    return -1

def check_data(target):
    test_data = [3, 2, 8, 9, 3, np.nan, 4, 7, 5]
    # We look for a zero in the data
    index = find_first(test_data, target)
    if index != -1:
        print('Data until first occurrence of', target, ':', test_data[:index])
    else:
        print('No occurrence of', target, 'in the data')

La fonction `find_first` trouve la première occurence d'un élément dans une liste (ou un tableau) et renvoie son indice. La fonction `check_data` utilise cette fonction pour trouver une certaine valeur dans les data et affiche les données jusqu'à ce point. Elle utilise des fausses données et prend comme argument `target` qui spécifie la valeur à trouver dans les données.

In [ ]:
check_data(5)

In [ ]:
check_data(1)

La commande `%debug` permet dans IPython ou dans le Jupyter Notebook d'accéder à la ligne de l'erreur. Elle lance la console `ipdb` (une interface améliorée du debugger intégré de Python `pdb`). On peut alors  accéder à l'état des variables à cette ligne.  
Parfois, les variables qui nous intéressent ne sont pas accéssibles à la ligne où l'exception a été levée. On peut alors se déplacer avec des commandes, telles que:
* `up` or `u`   
* `down` ou `d  
* `next` ou `n`
* `step` ou `s`: pour exécuter la ligne et en entrant dans les fonctions appelées dans cette ligne.  
* `continue` ou `c`  
* `quit` ou `q`

In [ ]:
%debug

#### Post mortem debugging avec pytest

On peut exécuter la suite de tests avec pytest et lui faire ouvrir le debugger quand il y a une erreur:

In [2]:
!cd exo_testing; py.test --pdb

====================================== test session starts =======================================
platform linux2 -- Python 2.7.6, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /home/camille/Documents/Cours/PythonMorpho/exo_testing, inifile: 
collected 0 items 

================================== no tests ran in 0.00 seconds ==================================


### 2.2 Debugging pas à pas

Regardons les fichiers `historgrams.py` et `do_equalization.py.` `historgrams.py` définit deux fonctions: `plot_histogram` et `equalize`. Le fonction `plot_histogram` dessine l'histogramme des valeurs dans un tabelau et la distribution cumulée de l'histograme. La fonction `equalize` réalise l'algorithme d'équalisation d'histogramme sur un image en niveau de gris.

In [3]:
!cat exo_testing/histograms.py

cat: exo_testing/histograms.py: No such file or directory


Cet algorithe permet de transformer l'image pour qu'elle ait un histogramme plat des valeurs d'intensité.

Essayons d'éxécuter `do_equalization.py` qui plotte un exemple d'image et son histogramme, l'équalise en utilisant la fonction ci-dessus et plotte le résultat:

In [4]:
%run exo_testing/do_equalization.py

ERROR: File `u'exo_testing/do_equalization.py'` not found.


Ca ne fonctionne pas... Les arguments de `np.interp` semblent ne pas avoir la même longueur. Utilisons le debugger pour examiner ce qui se passe:

In [ ]:
%debug

On dirait que `bins` et `cdf` que l'on donne à `np.interp` ont une longueur de différence. C'est quelque chose qui arrive souvent quand on travaille avec les histogrammes, car on utilise parfois des classes (N valeurs) et parfois des limites de classes (N+1 valeurs).    
Essayons de réparer celà en enlevant la dernière valeur de la variable `cdf`.

**Remarque sur Jupyter notebook et les fichiers externes:**  
Python n'importe un module qu'une seule fois. Du coup, si on édite une fonction dans un fichier, qui a déjà été importée, les changements ne seront pas pris en compte dans le notebook. Ce qu'on peut faire contre cela est de forcer le rechargement de tous les modules pour chaque execution de code:

In [ ]:
%load_ext autoreload
%autoreload 2

Bon, ça ne marche pas toujours, il faut parfois redémarrer le kernel du notebook et ré-exécuter les cellules.

In [ ]:
%run exo_testing/do_equalization.py

Hum... même si on n'obtient pas de message d'erreur, ça n'a pas l'air de bien marcher. On ne peut pas faire ici du "post mortem debugging", nous allons donc exécuter le script sous le contrôle du debugger avec la command:  
`%run -d do_equalization.py`.   
Hors du notebook, on peut faire appel à:   
`python -m pdb script.py`.

de même qu'avec `%debug`, on a un prompt, mais on peut maintenant exécuter le script. On peut utiliser les commandes mentionnées plus haut (`n`, `s`, `c`) pour naviguer dans l'exécution du fichier.

In [3]:
%run -d exo_testing/do_equalization.py

Breakpoint 1 at /home/camille/Documents/Cours/PythonMorpho/do_equalization.py:1
NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> /home/camille/Documents/Cours/PythonMorpho/do_equalization.py(1)<module>()
1---> 1 import matplotlib.pyplot as plt
      2 
      3 import histograms

ipdb> b (histogram:20)
***  '(histogram' not found from sys.path
ipdb> b histograms.py:20
*** Blank or comment
ipdb> break histograms:20
*** Blank or comment
ipdb> break histograms.py:20
*** Blank or comment
ipdb> q


## 3. Profiling

Un petit rappel sur l'approche générale:  
1. Vérifier que le code est correct  
2. Ecrire des tests pour être sûr que le code sera toujours correct après la phase d'optimisation  
3. Mesurer le temps total d'exécution et décider si on a besoin d'optimiser le code.  
4. Profiler le code pour décider où faire l'optimisation.  
5. Optimiser et réitérer au besoin.

<center>
<figure>
<img src='img/is_it_worth_the_time.png' width=400>
<figcaption>Is it worth the time? XKCD comic, licensed CC BY-NC 2.5</figcaption>
</figure>
</center>

### 3.1 Mesurer le temps total d'exécution  

IPython fournit deux commandes:
* `%time` mesure le temps total d'exécution de façon simple (comme la commande `time` du Unix shell). On peut donc l'utiliser si la commande/script a un long temps d'exécution.  
* `%timeit` répète les mesures plusieurs fois (3 par defaut) et chaque essai exécute la commande N fois (N est choisi pour que l'éxécution dure quelques secondes). 

In [9]:
import numpy as np
square_ar = np.random.rand(1000, 1000)
%time w, v = np.linalg.eig(square_ar)

CPU times: user 14.6 s, sys: 24.1 s, total: 38.8 s
Wall time: 10 s


In [10]:
%timeit square_ar.var()

100 loops, best of 3: 3.96 ms per loop


### 3.2 Mesurer plus en détails...

Les commandes ci-dessus nous aide à décider s'il s'avère utile d'optimiser notre code ou pas, mais elles ne nous donnent pas de renseignements sur quelles parties du code sont à optimiser.  

Comme exemple, on va utiliser la classique séquence de Fibonaci, où chaque élément est la somme des deux précédents éléments. On peut écrire cela avec une fonction récurcive (qui va avoir des temps d'exécution dramatiques quand `n` augmente...):  

In [11]:
def fibonacci(n):
    if n < 2:
        return n  # fibonacci(0) == 0, fibonacci(1) == 1
    else:
        return fibonacci(n - 2) + fibonacci(n - 1)

In [12]:
%time fibonacci(10)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 62 µs


55

In [13]:
%time fibonacci(20)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.08 ms


6765

In [15]:
%time fibonacci(35)

CPU times: user 4.18 s, sys: 48 ms, total: 4.23 s
Wall time: 4.19 s


9227465

Pour avoir une idée de ce qu'il se passe, on peut utiliser `%prun` qui fait appel au profiler intégré de Python.  
Hors de IPython/notebook, on peut utiliser:  
`python python -m cProfile [-o output_file] [-s sort_order] myscript.py`.

In [16]:
!python -m cProfile --help

Usage: cProfile.py [-o output_file_path] [-s sort] scriptfile [arg] ...

Options:
  -h, --help            show this help message and exit
  -o OUTFILE, --outfile=OUTFILE
                        Save stats to <outfile>
  -s SORT, --sort=SORT  Sort order when printing to stdout, based on pstats.Stats class


In [17]:
%prun fibonacci(35)

Cela nous renvoie trois types d'information pour chaque fonction appelée pendant l'exécution:
* `ncalls`: le nombre de fois que la fonction a été apellée  
* `tottime`: le temps total passé dans cette fonction  
* `cumtime`: le temps passé dans cette fonction, y compris le temps passé dans les fonctions appelées par cette fonction.  

De façon non surprenante, tout le temps est passé dans `fibonacci`, la fonction est appelé un grand nombre de fois!

Avez vous une idée pour programmer une fonction `fibonacci` un plus performante?

In [18]:
def fib(n):
    a,b = 1,1
    for i in range(n - 1):
        a, b = b, a + b
    return a

Pour des gros projets, il peut être pratique d'utiliser un outil graphique qui représente les informations de profiling de façon plus ergonomique.  
Un de ces outils est `snakeviz`.  
Sous IPython/notebook, on tape:
<code>
%load_ext snakeviz
%snakeviz fib(10)
</code>
Sinon:
<code>
$ python -m cProfile -o filename myscript.py
$ snakeviz filename
</code>

In [20]:
%load_ext snakeviz

In [21]:
%snakeviz fib(35)

 
*** Profile stats marshalled to file u'/tmp/tmpJPO_DZ'. 


In [22]:
%snakeviz fibonacci(35)

 
*** Profile stats marshalled to file u'/tmp/tmpSEbwKh'. 


`%prun` et `snakeviz` nous indiquent les fonctions qui prennent le plus de temps, mais n'aident pas à savoir quel est le potentiel d'optimisation dans les fonctions elle-mêmes.  

Un autre type de profiling est le **line-based profiling**, qui mesure le temps d'exécution de chaque ligne dans une ou plusieurs fonctions. Cette fonctionnalité est fournit par le **package `line_profiler`**. 
Sous IPython/notebook, on tape: 
<code>
%load_ext lprun
%lprun -f fonction_name fonction_name(20)
</code>
Sinon, il faut d'abord annoter la fonction avec le décorateur `@profile`. Ensuite il faut utiliser `kernprof` au lieu de python pour exécuter le script (car `python` ne connait pas `@profile`). Cela va créer un fichier `myscript.py.lprof` qui peut être afficher avec python.
<code>
$ kernprof -l myscript.py
$ python -m line_profiler myscript.py.lprof
</code>

In [25]:
%load_ext line_profiler

In [27]:
%lprun -f fib fib(35)